In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import datatable as dt
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [ ]:
train = dt.fread("../input/tabular-playground-series-sep21-processed-data/Tabular Playground_Series_Sep_2021_train_processed_data.csv").to_pandas()
test = dt.fread("../input/tabular-playground-series-sep21-processed-data/Tabular Playground_Series_Sep_2021_test_processed_data.csv").to_pandas()
sample= dt.fread("../input/tabular-playground-series-sep-2021/sample_solution.csv").to_pandas()

In [ ]:
train.head()

In [ ]:
train.isna().sum()

In [ ]:
test.isna().sum()

In [ ]:
from tqdm import tqdm
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score,confusion_matrix,classification_report
from sklearn.preprocessing import RobustScaler,QuantileTransformer
from sklearn.model_selection import StratifiedKFold

In [ ]:
features = [col for col in train.columns if col not in ['claim', 'id','n_missing','C0']]

In [ ]:
test_features= [col for col in test.columns if col not in ['id','n_missing','C0']]

In [ ]:
Xtrain = train[features].copy()
Ytrain = train['claim'].copy()
Xtest = test[test_features].copy()

print(f"Xtrain: {Xtrain.shape} \nYtrain: {Ytrain.shape} \nXtest: {Xtest.shape}")


In [ ]:
Xtest

In [ ]:
params = {}
params['objective']='binary:logistic'
params['eval_metric']='auc'
params['tree_method']='gpu_hist'
params['gpu_id']=0
params['predictor']='gpu_predictor'
params['n_estimators']=8000
params['max_depth']=4 
#params['learning_rate']=0.02
params['gamma']=0.2465
params['subsample']=0.6423
params['colsample_bytree']=0.775
params['colsample_bylevel']=0.868
params['min_child_weight']=366
params['reg_lambda']=0.5
params['reg_alpha']=10
params['verbosity']=0 
params['random_state']=20

In [ ]:
FOLD = 10
SEEDS = [24, 42]

fet_imp = 0
counter = 0
oof_score = 0
y_pred_final_xgb = np.zeros((Xtest.shape[0], 1))
y_pred_meta_xgb = np.zeros((Xtrain.shape[0], 1))


for sidx, seed in enumerate(SEEDS):
    seed_score = 0
    
    kfold = StratifiedKFold(n_splits=FOLD, shuffle=True, random_state=seed)

    for idx, (train, val) in enumerate(kfold.split(Xtrain.values, Ytrain.values)):
        counter += 1

        train_x, train_y = Xtrain.iloc[train], Ytrain.iloc[train]
        val_x, val_y = Xtrain.iloc[val], Ytrain.iloc[val]
        
        params['learning_rate']=0.001
        init_model = XGBClassifier(**params)

        init_model.fit(train_x, train_y, eval_set=[(train_x, train_y), (val_x, val_y)], 
                       early_stopping_rounds=200, verbose=500)
        
        params['learning_rate']=0.003
        model = XGBClassifier(**params)

        model.fit(train_x, train_y, eval_set=[(train_x, train_y), (val_x, val_y)], 
                  early_stopping_rounds=100, verbose=300, xgb_model=init_model)
        
        y_pred = model.predict_proba(val_x, iteration_range=(0, model.best_iteration))[:,-1]
        y_pred_meta_xgb[val] += np.array([y_pred]).T
        y_pred_final_xgb += np.array([model.predict_proba(Xtest, iteration_range=(0, model.best_iteration))[:,-1]]).T
        
        fet_imp += model.feature_importances_
        score = roc_auc_score(val_y, y_pred)
        oof_score += score
        seed_score += score
        print("\nSeed-{} | Fold-{} | OOF Score: {}\n".format(seed, idx, score))
    
    print("\nSeed: {} | Aggregate OOF Score: {}\n\n".format(seed, (seed_score / FOLD)))


fet_imp = fet_imp / float(counter)
y_pred_meta_xgb = y_pred_meta_xgb / float(len(SEEDS))
y_pred_final_xgb = y_pred_final_xgb / float(counter)
oof_score /= float(counter)
print("Aggregate OOF Score: {}".format(oof_score))

In [ ]:
'''
def plot_confusion_matrix(cm, classes):

    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion matrix', fontweight='bold', pad=15)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label', fontweight='bold')
    plt.xlabel('Predicted label', fontweight='bold')
    plt.tight_layout()
    
'''


In [ ]:
# cnf_matrix = confusion_matrix(Ytrain, y_pred, labels=[0, 1])
# np.set_printoptions(precision=2)
# plt.figure(figsize=(12, 5))
# plot_confusion_matrix(cnf_matrix, classes=[0, 1])

In [ ]:


submit_df = pd.read_csv("../input/tabular-playground-series-sep-2021/sample_solution.csv")
submit_df['claim'] = y_pred_final_xgb
submit_df.to_csv("XGB_Submission1.csv", index=False)
submit_df.head(10)

